# LlamaIndex + MCP Usage

The `llama-index-tools-mcp` package provides several tools for using MCP with LlamaIndex.

In [ ]:
%pip install llama-index-tools-mcp

## Using Tools from an MCP Server

Using the `get_tools_from_mcp_url` or `aget_tools_from_mcp_url` function, you can get a list of `FunctionTool`s from an MCP server.

In [ ]:
from llama_index.tools.mcp import (
    get_tools_from_mcp_url,
    aget_tools_from_mcp_url,
)

# async
tools = await aget_tools_from_mcp_url("http://127.0.0.1:8000/sse")

By default, this will use our `BasicMCPClient`, which will run a command or connect to the URL and return the tools.

You can also pass in a custom `ClientSession` to use a different client.

You can also specify a list of allowed tools to filter the tools that are returned.

In [ ]:
from llama_index.tools.mcp import BasicMCPClient

client = BasicMCPClient("http://127.0.0.1:8000/sse")

tools = await aget_tools_from_mcp_url(
    "http://127.0.0.1:8000/sse",
    client=client,
    allowed_tools=["tool1", "tool2"],
)

## Converting a Workflow to an MCP App

If you have a custom `Workflow`, you can convert it to an MCP app using the `workflow_as_mcp` function.

For example, let's use the following workflow that will make a string loud:

In [ ]:
from llama_index.core.workflow import (
    Context,
    Workflow,
    Event,
    StartEvent,
    StopEvent,
    step,
)
from llama_index.tools.mcp.utils import workflow_as_mcp


class RunEvent(StartEvent):
    msg: str


class InfoEvent(Event):
    msg: str


class LoudWorkflow(Workflow):
    """Useful for converting strings to uppercase and making them louder."""

    @step
    def step_one(self, ctx: Context, ev: RunEvent) -> StopEvent:
        ctx.write_event_to_stream(InfoEvent(msg="Hello, world!"))

        return StopEvent(result=ev.msg.upper() + "!")


workflow = LoudWorkflow()

mcp = workflow_as_mcp(workflow)

This code will automatically generate a `FastMCP` server that will
- Use the workflow class name as the tool name
- Use our custom `RunEvent` as the typed inputs to the tool
- Automatically use the SSE stream for streaming json dumps of the workflow event stream

If this code was in a script called `script.py`, you could launch the MCP server with:

```bash
mcp dev script.py
```

Or the other commands documented in the [MCP CLI README](https://github.com/modelcontextprotocol/python-sdk?tab=readme-ov-file#quickstart).

Note that to launch from the CLI, you may need to install the MCP CLI:

```bash
pip install "mcp[cli]"
```



You can further customize the `FastMCP` server by passing in additional arguments to the `workflow_as_mcp` function:
- `workflow_name`: The name of the workflow. Defaults to the class name.
- `workflow_description`: The description of the workflow. Defaults to the class docstring.
- `start_event_model`: The event model to use for the start event. You can either use a custom `StartEvent` class in your workflow or pass in your own pydantic model here to define the inputs to the workflow.
- `**fastmcp_init_kwargs`: Any extra arguments to pass to the `FastMCP()` server constructor.